In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'kaggle_kkbox_churn_220804.ipynb',
 'kaggle_kkbox_churn_220804.zip',
 'kaggle_kkbox_churn_220805.ipynb',
 'members_v3.csv',
 'train_v2.csv',
 'transactions_v2.csv',
 'user_logs_v2.csv']

In [4]:
df_mem = pd.read_csv("members_v3.csv", low_memory = False)
df_tr = pd.read_csv("train_v2.csv", low_memory = False)

In [5]:
df_mem.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time'],
      dtype='object')

In [6]:
df_tr.columns

Index(['msno', 'is_churn'], dtype='object')

In [7]:
df_tr.head(1)

msno  is_churn
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1

In [10]:
df_join = pd.merge(df_tr, df_mem, on = "msno", how = "inner")
len(df_join)

860967

In [11]:
print(len(df_mem))
print(len(df_tr))

6769473
970960


In [12]:
df_tr["is_churn"].value_counts(normalize = True).round(3)

0    0.91
1    0.09
Name: is_churn, dtype: float64

In [13]:
df_join["is_churn"].value_counts(normalize = True).round(3)

0    0.905
1    0.095
Name: is_churn, dtype: float64

종속변수 불균형(imblanced data)
 - over sampling
 - under sampling

In [14]:
df_join.head(2)

msno  is_churn  city  bd gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28   male   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20   male   

   registered_via  registration_init_time  
0               3                20131223  
1               3                20131223

In [21]:
df_join.columns = list(df_join.columns[:-2]) + ["reg", "date"]
df_join.head(2)

msno  is_churn  city  bd gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28   male   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20   male   

   reg      date  
0    3  20131223  
1    3  20131223

In [22]:
df_join["gender"].isna().sum()

472062

In [23]:
df_join["gender"].unique()

array(['male', nan, 'female'], dtype=object)

In [24]:
df_join["gender"] = df_join["gender"].fillna(-1)
df_join["gender"].unique()

array(['male', -1, 'female'], dtype=object)

In [25]:
df_join["gender"] = df_join["gender"].replace(dict(male = 1,
                                                   female = 2))
df_join["gender"].unique()

array([ 1, -1,  2], dtype=int64)

In [26]:
df_join.head(2)

msno  is_churn  city  bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28       1   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20       1   

   reg      date  
0    3  20131223  
1    3  20131223

In [27]:
df_join.to_csv("main.csv", index = False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("main.csv")
df.head(2)

msno  is_churn  city  bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28       1   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20       1   

   reg      date  
0    3  20131223  
1    3  20131223

In [45]:
df_tr = pd.read_csv("transactions_v2.csv")
df_tr.head(1)

msno  payment_method_id  \
0  ++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=                 32   

   payment_plan_days  plan_list_price  actual_amount_paid  is_auto_renew  \
0                 90              298                 298              0   

   transaction_date  membership_expire_date  is_cancel  
0          20170131                20170504          0

In [11]:
len(df_tr)

In [12]:
df_tr["msno"].nunique()

In [10]:
df_tr_sub = df_tr.loc[df_tr["msno"].isin(df_tr["msno"].unique()[15:17]), ]
# df_tr_sub

### Q1. 사용자별 결제시도 횟수를 구하시오.
※ df_tr_sub 객체를 대상으로 분석하시오.

In [15]:
df_tr_sub["cnt"] = 1
df_tr_sub.groupby("msno")["cnt"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=     1
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    20
Name: cnt, dtype: int64

In [16]:
df_tr_sub["msno"].value_counts()

+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    20
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=     1
Name: msno, dtype: int64

### Q2. 사용자별 주 결제방법을 구하시오.
※ df_tr_sub 객체를 대상으로 분석하시오.

In [21]:
df_tr_sub.groupby("msno")["payment_method_id"].agg(lambda x: x.mode())

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    41
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    41
Name: payment_method_id, dtype: int64

In [40]:
aa = df_tr_sub.groupby("msno")["payment_method_id"].value_counts()
aa = aa.to_frame()
aa.columns = ["cnt"]
aa = aa.reset_index().sort_values("cnt", ascending = False)
aa.groupby("msno").head(1).reset_index(drop = True)

msno  payment_method_id  cnt
0  +3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=                 41   20
1  +2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=                 41    1

### Q3. 결제 상품의 기간을 기준으로 사용자별 결제한 상품의 종류를 계산하시오.
※ 30일 상품만 결제하면 1 / 30일, 7일, 90일 상품을 결제했으면 3

In [43]:
df_tr_sub.groupby("msno")["payment_plan_days"].nunique()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    1
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    2
Name: payment_plan_days, dtype: int64

### Q4. 결제 상품의 기간을 모두 합산하시오.
※ 단, is_cancel 을 참고하여 취소분은 제외하시오.

In [47]:
df_tr_sub2 = df_tr_sub.loc[df_tr_sub["is_cancel"] == 0, ]
df_tr_sub2.groupby("msno")["payment_plan_days"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=     30
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    540
Name: payment_plan_days, dtype: int64

### Q5. 7일 이하의 상품 결제 여부를 산출하시오.

In [50]:
df_tr_sub.groupby("msno")["payment_plan_days"].agg(lambda x: sum(x <= 7))

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    0
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    1
Name: payment_plan_days, dtype: int64

### Q6. plan_list_price와 actual_amount_paid 변수를 참고하여 사용자별 실제 지불 총액과 할인된 가격으로 상품을 결제한 횟수를 계산하시오.

In [51]:
df_tr_sub.groupby("msno")["actual_amount_paid"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=      99
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    2931
Name: actual_amount_paid, dtype: int64

In [53]:
df_tr_sub["cnt_dis"] = (df_tr_sub["plan_list_price"] > df_tr_sub["actual_amount_paid"]) + 0
df_tr_sub.groupby("msno")["cnt_dis"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    0
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    0
Name: cnt_dis, dtype: int32

### Q7. 사용자별 자동갱신 결제 여부를 계산하시오.

In [55]:
df_tr_sub.groupby("msno")["is_auto_renew"].max()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    1
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    1
Name: is_auto_renew, dtype: int64

### Q8. 사용자별 거래 취소 경험 여부를 계산하시오.

In [57]:
df_tr_sub.groupby("msno")["is_cancel"].max()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    0
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    1
Name: is_cancel, dtype: int64

### Q9. Q1 부터 Q8까지 했던 작업을 df_tr 객체를 대상으로 실시하고 그 결과를 전부 취합하여 하나의 데이터프레임으로 정리하시오.
※ 최종 정리 객체명은 df_tr_agg 로 하시오.   
※ df_tr_agg 객체의 row 개수는 1197050 이어야 한다.  
※ Q1: 사용자별 결제시도 횟수  
※ Q2: 사용자별 주 결제방법  
※ Q3: 사용자별 결제한 상품의 종류  
※ Q4: 결제 상품의 기간  
※ Q5: 7일 이하의 상품 결제 여부  
※ Q6: 사용자별 실제 지불 총액과 할인된 가격으로 상품을 결제한 횟수  
※ Q7: 사용자별 자동갱신 결제 여부  
※ Q8: 사용자별 거래 취소 경험 여부

In [58]:
df_tr["msno"].nunique()

1197050

In [59]:
df_tr["cnt"] = 1

In [69]:
df_tr_agg = df_tr.groupby("msno")["cnt"].sum().reset_index()
df_tr_agg.head(2)

msno  cnt
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1

In [70]:
df_tr_q2 = df_tr.groupby("msno")["payment_method_id"].agg(lambda x: x.mode())
df_tr_q2 = df_tr_q2.reset_index()

In [71]:
df_tr_q2.head(2)

msno payment_method_id
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=                22
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=                41

In [72]:
df_tr_agg = df_tr_agg.merge(df_tr_q2, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41

In [73]:
df_tr_q3 = df_tr.groupby("msno")["payment_plan_days"].nunique()
df_tr_q3 = df_tr_q3.reset_index()
df_tr_q3.columns = ["msno", "cnt_p_type"]
df_tr_agg = df_tr_agg.merge(df_tr_q3, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  
0           1  
1           1

In [74]:
df_tr_q4 = df_tr.loc[df_tr["is_cancel"] == 0, ]
df_tr_q4 = df_tr_q4.groupby("msno")["payment_plan_days"].sum()
df_tr_q4 = df_tr_q4.reset_index()
df_tr_q4.columns = ["msno", "p_day_sum"]
df_tr_agg = df_tr_agg.merge(df_tr_q4, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  
0           1      395.0  
1           1       30.0

In [81]:
df_tr_sub = df_tr.loc[df_tr["is_cancel"] == 1, ]
df_tr_sub.head(2)

msno  payment_method_id  \
3    +/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=                 36   
177  /PQk/sPb+NaF4URt2hsJjN99n/RZ05J78CP32jVx8qk=                 41   

     payment_plan_days  plan_list_price  actual_amount_paid  is_auto_renew  \
3                   30              180                 180              1   
177                 30               99                  99              1   

     transaction_date  membership_expire_date  is_cancel  cnt  cnt_dis  
3            20170329                20170331          1    1        0  
177          20150708                20171105          1    1        0

In [83]:
df_tr_sub_c = df_tr.loc[df_tr["msno"].isin(df_tr_sub["msno"]), ]
df_tr_msno_cnt = df_tr_sub_c["msno"].value_counts().reset_index()
df_tr_msno_cnt.loc[df_tr_msno_cnt["msno"] == 1, ]

index  msno
24176  KX+988+/GbcIUwwH5hwi41PROvbEReL34kKEmX1mr9g=     1
24177  XBR6tmgEsMwFez+hU9ERfCPZ4Jxzt8ohgDZmAhvzmGs=     1
24178  MQUkOsIGoKyl2PTLpjLDaG7HVKaQnWvEAHY6W0xbzWo=     1
24179  NCCIxdA8nzCrsVhFO56y3nuLRWqrxtmegZSeFAoeBZk=     1
24180  EQeclepjh08CP+JMN+sseJ8UXIC5B3TAV89A8LtcSe8=     1
...                                             ...   ...
34546  l998p1WAKXMgetk+GCB7MIQIha+atSFKnltYICib2QM=     1
34547  nDRnFV+oMRWbxxXI+b3DpaQn/dCHp0KB5Wm+4b1y17Q=     1
34548  neAvmbQyKhcHLeix6McavenFhNKGVi+2SagYvBdmFyY=     1
34549  oEgl1XF4dBMSqNWS31w8bdmrR68i0Q1ZPkLK0gJUUeA=     1
34550  ymghcYW9EE6VigAFNrdBZNJ+NLNly2FpM35K8mB63n0=     1

[10375 rows x 2 columns]

In [75]:
df_tr_q5 = df_tr.groupby("msno")["payment_plan_days"].agg(lambda x: sum(x <= 7))
df_tr_q5 = df_tr_q5.reset_index()
df_tr_q5.columns = ["msno", "cnt_p_7LE"] # Less than Equal
df_tr_agg = df_tr_agg.merge(df_tr_q5, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  
0           1      395.0          0  
1           1       30.0          0

In [76]:
df_tr_q6 = df_tr.groupby("msno")["actual_amount_paid"].sum()
df_tr_q6 = df_tr_q6.reset_index()
df_tr_q6.columns = ["msno", "sum_p"]
df_tr_agg = df_tr_agg.merge(df_tr_q6, on = "msno", how = "left")

df_tr["cnt_dis"] = (df_tr["plan_list_price"] > df_tr["actual_amount_paid"]) + 0
df_tr_q6 = df_tr.groupby("msno")["cnt_dis"].sum()
df_tr_q6 = df_tr_q6.reset_index()
df_tr_q6.columns = ["msno", "cnt_p_dis"]
df_tr_agg = df_tr_agg.merge(df_tr_q6, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  sum_p  cnt_p_dis  
0           1      395.0          0   1599          0  
1           1       30.0          0     99          0

In [77]:
df_tr_q7 = df_tr.groupby("msno")["is_auto_renew"].max()
df_tr_q7 = df_tr_q7.reset_index()
df_tr_q7.columns = ["msno", "is_p_ar"]
df_tr_agg = df_tr_agg.merge(df_tr_q7, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  sum_p  cnt_p_dis  is_p_ar  
0           1      395.0          0   1599          0        0  
1           1       30.0          0     99          0        1

In [78]:
df_tr_q8 = df_tr.groupby("msno")["is_cancel"].max()
df_tr_q8 = df_tr_q8.reset_index()
df_tr_q8.columns = ["msno", "is_p_cancel"]
df_tr_agg = df_tr_agg.merge(df_tr_q8, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  sum_p  cnt_p_dis  is_p_ar  is_p_cancel  
0           1      395.0          0   1599          0        0            0  
1           1       30.0          0     99          0        1            0

In [79]:
len(df_tr_agg)

1197050

In [80]:
df_tr_agg.isna().sum().reset_index()

index      0
0               msno      0
1                cnt      0
2  payment_method_id      0
3         cnt_p_type      0
4          p_day_sum  10376
5          cnt_p_7LE      0
6              sum_p      0
7          cnt_p_dis      0
8            is_p_ar      0
9        is_p_cancel      0

In [84]:
df_tr_agg = df_tr_agg.fillna(0)
df_tr_agg.isna().sum().reset_index()

index  0
0               msno  0
1                cnt  0
2  payment_method_id  0
3         cnt_p_type  0
4          p_day_sum  0
5          cnt_p_7LE  0
6              sum_p  0
7          cnt_p_dis  0
8            is_p_ar  0
9        is_p_cancel  0

In [ ]:
df_tr_agg.to_csv("transactions_agg.csv", index = False)

-----------------------------
# user_log.csv

In [86]:
df_log = pd.read_csv("user_logs_v2.csv")
df_log.head(2)

msno      date  num_25  num_50  \
0  u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=  20170331       8       4   
1  nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=  20170330       2       2   

   num_75  num_985  num_100  num_unq  total_secs  
0       0        1       21       18    6309.273  
1       1        0        9       11    2390.699

In [87]:
df_log["date"].quantile([0, 1])

0.0    20170301.0
1.0    20170331.0
Name: date, dtype: float64

### Q1. date 변수를 활용하여 사용자별 3월 스트리밍 서비스 이용율을 계산하시오.

In [96]:
df_log_agg = df_log.groupby("msno")["date"].nunique().to_frame()
df_log_agg["date"] = df_log_agg["date"] / 31
df_log_agg.head(2)

date       mean  max
msno                                                                 
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=  0.83871  22.461538  141
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=  1.00000  35.129032   84

### Q2. 각 음악 재생 길이 변수를 활용하여 사용자별 일별 총 재생 음악숫자를 계산하고 해당 계산 결과를 기반으로 일평균 재생 음악 숫자와 일최대 재생 음악 숫자를 산출하시오.
※ 관련 변수는 num_25, num_50, num_75, num_985, num_100 이다.

In [91]:
df_log["cnt_m_total"] = df_log.loc[:, "num_25":"num_100"].sum(axis = 1)

In [94]:
df_log_q2 = df_log.groupby("msno")["cnt_m_total"].agg(["mean", "max"])
df_log_q2.head(2)

mean  max
msno                                                        
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=  22.461538  141
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=  35.129032   84

In [97]:
df_log_agg = df_log_agg.join(df_log_q2, how = "left")
df_log_agg.head(2)

date       mean  max
msno                                                                 
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=  0.83871  22.461538  141
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=  1.00000  35.129032   84

In [98]:
df_log_agg.columns = ["m_play_r", "m_u_mean", "m_u_max"]
df_log_agg.head(1)

m_play_r   m_u_mean  m_u_max
msno                                                                      
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=   0.83871  22.461538      141

### Q3. 각 음악 재생 길이 변수를 활용하여 사용자별 일별 음악을 완전히 듣지 않고 넘겨버린 음악 개수를 기반으로 음악 불완전 재생 지표를 생성하시오.
※ 변수별 가중치: 4, 3, 2, 1, 0

In [ ]:
df_log["m_p_idx"] = (df_log["num_25"] * 4) + (df_log["num_50"] * 3) \
 + (df_log["num_75"] * 2) + df_log["num_985"]

### Q4. 불완전 재생지표를 사용자별로 평균과 최대값을 산출하시오.

In [109]:
df_log_q4 = df_log.groupby("msno")["m_p_idx"].agg(["mean", "max"])
df_log_q4.columns = ["m_p_idx_mean", "m_p_idx_max"]
df_log_q4.head(2)

m_p_idx_mean  m_p_idx_max
msno                                                                   
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=     15.461538           42
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=     42.838710          122

In [110]:
df_log_agg = df_log_agg.join(df_log_q4, how = "left")

### Q5. 사용자별 일별 음악 재생시간 변수의 단위를 분으로 바꾸고 반올림하여 1의 자리까지 계산한 내용을 "m_tm" 변수에 저장하고. 해당 변수를 기준으로 사용자별 평균/최대값/표준편차를 계산하시오
※ 최종 변수명은 m_tm_mean/m_tm_max/m_tm_std 로 지정하시오.

In [112]:
df_log["m_tm"] = round(df_log["total_secs"] / 60)
df_log_q5 = df_log.groupby("msno")["m_tm"].agg(["mean", "max", "std"])
df_log_q5.columns = ["m_tm_mean", "m_tm_max", "m_tm_std"]
df_log_agg = df_log_agg.join(df_log_q5, how = "left")

In [113]:
df_log_agg.head(2)

m_play_r   m_u_mean  m_u_max  \
msno                                                                         
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=   0.83871  22.461538      141   
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=   1.00000  35.129032       84   

                                              m_p_idx_mean  m_p_idx_max  \
msno                                                                      
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=     15.461538           42   
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=     42.838710          122   

                                               m_tm_mean  m_tm_max    m_tm_std  
msno                                                                            
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=   75.576923     529.0  107.945977  
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=  103.516129     237.0   56.360667

In [114]:
df_log_agg.isna().sum()

m_play_r            0
m_u_mean            0
m_u_max             0
m_p_idx_mean        0
m_p_idx_max         0
m_tm_mean           0
m_tm_max            0
m_tm_std        87222
dtype: int64

In [115]:
df_log_agg = df_log_agg.fillna(0).reset_index()
df_log_agg.head(2)

msno  m_play_r   m_u_mean  m_u_max  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=   0.83871  22.461538      141   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=   1.00000  35.129032       84   

   m_p_idx_mean  m_p_idx_max   m_tm_mean  m_tm_max    m_tm_std  
0     15.461538           42   75.576923     529.0  107.945977  
1     42.838710          122  103.516129     237.0   56.360667

In [116]:
df_log_agg.to_csv("user_logs_agg.csv", index = False)

-----------------------------------------
# main

In [119]:
df_mem = pd.read_csv("members_v3.csv")
df_mem.head(1)

msno  city  bd gender  \
0  Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=     1   0    NaN   

   registered_via  registration_init_time  
0              11                20110911

In [123]:
df_cnt_city = df_mem["city"].value_counts().reset_index()
df_cnt_city["city_lv"] = pd.cut(df_cnt_city["city"],
                    bins = [0, 100000, 200000, 300000, 400000, 5000000],
                               labels = [0, 1, 2, 3, 4])
# df_cnt_city.head(10)
df_cnt_city.to_csv("member_city_info.csv", index = False)

In [133]:
df_cnt_reg = df_mem["registered_via"].value_counts().reset_index()
df_cnt_reg["reg_lv"] = pd.cut(df_cnt_reg["registered_via"],
                              bins = [0, 1e6, 2e6, 3e6],
                              labels = [0, 1, 2])
# df_cnt_reg.head(10)
df_cnt_reg.to_csv("member_reg_via_info.csv", index = False)

In [153]:
df = pd.read_csv("main.csv")
df["bd"] = round((df["bd"] + 3152) / 365)
df["date"] = pd.to_datetime(df["date"], format = "%Y%m%d")
df.head(2)

msno  is_churn  city   bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  9.0       1   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  9.0       1   

   reg       date  
0    3 2013-12-23  
1    3 2013-12-23

In [145]:
(pd.to_datetime("2017-04-01") - df["date"][:3]).dt.days / 30

0    39.833333
1    39.833333
2    39.700000
Name: date, dtype: float64

In [154]:
df["join_month"] = (pd.to_datetime("2017-04-01") - df["date"]).dt.days / 30
df.head(2)

msno  is_churn  city   bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  9.0       1   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  9.0       1   

   reg       date  join_month  
0    3 2013-12-23   39.833333  
1    3 2013-12-23   39.833333

In [155]:
df = df.merge(df_cnt_city.drop("city", axis = 1),
              left_on = "city", right_on = "index")
df = df.merge(df_cnt_reg.drop("registered_via", axis = 1),
              left_on = "reg", right_on = "index")

In [157]:
df2 = df.drop(["city", "reg", "date", "index_x", "index_y"], axis = 1)
df2.head(2)

msno  is_churn   bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1  9.0       1   
1  z+DiUWa91dI277IJDsJLTTe61ho3oRnUG/0UXS+y8Mk=         1  9.0       2   

   join_month city_lv reg_lv  
0   39.833333       3      1  
1   32.900000       3      1

In [158]:
df2.to_csv("main_v2.csv", index = False)

-----------------------
# join

In [1]:
import pandas as pd

In [9]:
df = pd.read_csv("main_v2.csv")
df_tr = pd.read_csv("transactions_agg.csv")
df_log = pd.read_csv("user_logs_agg.csv")

In [10]:
df = df.merge(df_tr, on = "msno", how = "left")
df = df.merge(df_log, on = "msno", how = "left")

In [11]:
len(df)

860967

In [5]:
df.head(1)

msno  is_churn   bd  gender  \
0  2l0VkVyZk8+TEN2fiOfufZjqZ69MGmLL4hS3VBXtOHY=         1  9.0       1   

   join_month  city_lv  reg_lv  cnt payment_method_id  cnt_p_type  ...  \
0   19.766667        3       1    3                36           1  ...   

   is_p_ar  is_p_cancel  m_play_r  m_u_mean  m_u_max  m_p_idx_mean  \
0        1            0  0.096774  6.666667       18           0.0   

   m_p_idx_max  m_tm_mean  m_tm_max   m_tm_std  
0            0  32.333333      87.0  47.342722  

[1 rows x 24 columns]

In [13]:
df.isna().sum()

msno                      0
is_churn                  0
bd                        0
gender                    0
join_month                0
city_lv                   0
reg_lv                    0
cnt                   35599
payment_method_id     35599
cnt_p_type            35599
p_day_sum             35599
cnt_p_7LE             35599
sum_p                 35599
cnt_p_dis             35599
is_p_ar               35599
is_p_cancel           35599
m_play_r             106435
m_u_mean             106435
m_u_max              106435
m_p_idx_mean         106435
m_p_idx_max          106435
m_tm_mean            106435
m_tm_max             106435
m_tm_std             106435
dtype: int64

In [14]:
df = df.dropna()

In [15]:
df.to_csv("main_join_v1.csv", index = False)

-----------
# modeling

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [35]:
df = pd.read_csv("main_join_v1.csv")
df.head(2)

msno  is_churn   bd  gender  \
0  2l0VkVyZk8+TEN2fiOfufZjqZ69MGmLL4hS3VBXtOHY=         1  9.0       1   
1  dxsdoOAqkQ0rIQZ/sR0U++dFlmPj+r3xr849yRVy0Qs=         1  9.0       2   

   join_month  city_lv  reg_lv  cnt payment_method_id  cnt_p_type  ...  \
0   19.766667        3       1  3.0                36         1.0  ...   
1   57.066667        3       1  3.0                39         1.0  ...   

   is_p_ar  is_p_cancel  m_play_r   m_u_mean  m_u_max  m_p_idx_mean  \
0      1.0          0.0  0.096774   6.666667     18.0       0.00000   
1      1.0          0.0  0.741935  13.130435     33.0       3.73913   

   m_p_idx_max  m_tm_mean  m_tm_max   m_tm_std  
0          0.0  32.333333      87.0  47.342722  
1         33.0  49.956522     104.0  28.434100  

[2 rows x 24 columns]

In [39]:
df["payment_method_id"] = df["payment_method_id"].str.replace(pat="[^0-9 ]",
                                                              repl = "",
                                                              regex = True)
df["payment_method_id"] = df["payment_method_id"].str.split(" ")

In [42]:
df["payment_method_id"] = df["payment_method_id"].apply(lambda x: x[0])

In [49]:
pd.crosstab(df["payment_method_id"], df["is_churn"],
            normalize = 0).round(2)

is_churn              0     1
payment_method_id            
                   0.00  1.00
10                 0.90  0.10
11                 0.98  0.02
12                 0.03  0.97
13                 0.00  1.00
14                 0.98  0.02
15                 0.08  0.92
16                 0.82  0.18
17                 0.20  0.80
18                 0.97  0.03
19                 0.97  0.03
20                 0.00  1.00
21                 0.95  0.05
22                 0.00  1.00
23                 0.98  0.02
26                 0.35  0.65
27                 0.98  0.02
28                 0.79  0.21
29                 0.93  0.07
3                  0.00  1.00
30                 0.94  0.06
31                 0.99  0.01
32                 0.02  0.98
33                 0.97  0.03
34                 0.98  0.02
35                 0.04  0.96
36                 0.92  0.08
37                 0.98  0.02
38                 0.78  0.22
39                 0.95  0.05
40                 0.95  0.05
41                 0.97  0.03
6                  0.00  1.00
8                  0.00  1.00

In [51]:
df_pay = pd.crosstab(df["payment_method_id"], df["is_churn"],
                     normalize = 0).reset_index()
df_pay["p_id_lv"] = (df_pay[1] > 0.2) + 0
df_pay = df_pay[["payment_method_id", "p_id_lv"]]
df_pay.head(2)

is_churn payment_method_id  p_id_lv
0                                 1
1                       10        0

In [52]:
df = df.merge(df_pay, on = "payment_method_id", how = "left")
df = df.drop("payment_method_id", axis = 1)
df.head(2)

msno  is_churn   bd  gender  \
0  2l0VkVyZk8+TEN2fiOfufZjqZ69MGmLL4hS3VBXtOHY=         1  9.0       1   
1  dxsdoOAqkQ0rIQZ/sR0U++dFlmPj+r3xr849yRVy0Qs=         1  9.0       2   

   join_month  city_lv  reg_lv  cnt  cnt_p_type  p_day_sum  ...  is_p_cancel  \
0   19.766667        3       1  3.0         1.0       90.0  ...          0.0   
1   57.066667        3       1  3.0         1.0       90.0  ...          0.0   

   m_play_r   m_u_mean  m_u_max  m_p_idx_mean  m_p_idx_max  m_tm_mean  \
0  0.096774   6.666667     18.0       0.00000          0.0  32.333333   
1  0.741935  13.130435     33.0       3.73913         33.0  49.956522   

   m_tm_max   m_tm_std  p_id_lv  
0      87.0  47.342722        0  
1     104.0  28.434100        0  

[2 rows x 24 columns]

In [53]:
# df["payment_method_id"].unique()

In [54]:
df["is_churn"].value_counts(normalize = True)

0    0.935784
1    0.064216
Name: is_churn, dtype: float64

In [55]:
df_tr, df_te = train_test_split(df, train_size = 0.8,
                                random_state = 123,
                                stratify = df["is_churn"])
print(len(df_tr))
print(len(df_te))

580577
145145


In [56]:
df_tr["is_churn"].value_counts(normalize = True)

0    0.935785
1    0.064215
Name: is_churn, dtype: float64

In [59]:
X_colnames = ["bd", "gender", "join_month", "city_lv", "reg_lv"]

In [60]:
model1 = LogisticRegression(max_iter = 300)
model1.fit(X = df_tr[X_colnames],
           y = df_tr["is_churn"])

LogisticRegression(max_iter=300)

In [ ]:
from sklearn.metrics import f1_score

In [81]:
pred = model1.predict_proba(df_te[X_colnames])
pred_class = (pred[:, 1] > 0.2) + 0
print("f1: ", f1_score(df_te["is_churn"], pred_class))
print("auc: ", roc_auc_score(df_te["is_churn"], pred[:, 1]))
pd.crosstab(df_te["is_churn"], pred_class)

f1:  0.05281966355791858
auc:  0.6655661169533356


col_0          0     1
is_churn              
0         133975  1849
1           9018   303

In [85]:
X_col_v2 = X_colnames + ["cnt", "cnt_p_type", "p_day_sum", "cnt_p_7LE",
                         "sum_p", "cnt_p_dis", "is_p_ar", "is_p_cancel",
                         "p_id_lv"]

In [86]:
model2 = LogisticRegression(max_iter = 300)
model2.fit(X = df_tr[X_col_v2],
           y = df_tr["is_churn"])

C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=300)

In [96]:
pred = model2.predict_proba(df_te[X_col_v2])
pred_class = (pred[:, 1] > 0.8) + 0
print("f1: ", f1_score(df_te["is_churn"], pred_class))
print("auc: ", roc_auc_score(df_te["is_churn"], pred[:, 1]))
pd.crosstab(df_te["is_churn"], pred_class)

f1:  0.47191919191919196
auc:  0.907945059020383


col_0          0     1
is_churn              
0         135690   134
1           6401  2920

In [97]:
model3 = LogisticRegression(max_iter = 500)
model3.fit(X = df_tr.drop(["msno", "is_churn"], axis = 1),
           y = df_tr["is_churn"])

C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [100]:
pred = model3.predict_proba(df_te.drop(["msno", "is_churn"], axis = 1))
pred_class = (pred[:, 1] > 0.8) + 0
print("f1: ", f1_score(df_te["is_churn"], pred_class))
print("auc: ", roc_auc_score(df_te["is_churn"], pred[:, 1]))
pd.crosstab(df_te["is_churn"], pred_class)

f1:  0.4945928753180661
auc:  0.8681681326392351


col_0          0     1
is_churn              
0         135679   145
1           6211  3110